In [ ]:
# import patool
# Download CBDB sqlite
# !git clone https://github.com/cbdb-project/cbdb_sqlite.git
import requests
import os
import warnings
import urllib3

# 禁用所有 InsecureRequestWarning 警告
warnings.simplefilter('ignore', urllib3.exceptions.InsecureRequestWarning)

# url = "https://hf-mirror.com/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url, verify=False)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

In [ ]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
print(latest_file)
latest_db = latest_file.split("/")[-1].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

In [ ]:
import py7zr

# 指定压缩文件和目标目录
archive = './cbdb_sqlite/latest.7z'
output_dir = ''

# 解压文件
with py7zr.SevenZipFile(archive, 'r') as archive:
    archive.extractall(output_dir)

print("解压完成！")



In [1]:
import sqlite3
import json

# 连接数据库
conn = sqlite3.connect('latest.db')
cursor = conn.cursor()

# 指定所需的朝代及其对应的 c_dy 值
dynasties = [
    (6, 'Tang'),
    (15, 'Song'),
    (16, 'Liao'),
    (17, 'Jin'),
    (18, 'Yuan'),
    (19, 'Ming'),
    (20, 'Qing')
]

result = {}

for dy in dynasties:
    c_dy = dy[0]
    dynasty_name = dy[1]
    print(dy)

    # 查询每本书记录的人数
    query = f'''
    SELECT TEXT_CODES.c_title_chn, COUNT(DISTINCT BIOG_MAIN.c_personid) as person_count
    FROM BIOG_MAIN
    JOIN BIOG_SOURCE_DATA ON BIOG_MAIN.c_personid = BIOG_SOURCE_DATA.c_personid
    JOIN TEXT_CODES ON BIOG_SOURCE_DATA.c_textid = TEXT_CODES.c_textid
    WHERE BIOG_MAIN.c_dy = '{c_dy}'
    GROUP BY TEXT_CODES.c_title_chn
    ORDER BY person_count DESC
    LIMIT 20
    '''
    cursor.execute(query)
    books = cursor.fetchall()

    # 将结果存入字典
    result[dynasty_name] = [{'title': book[0], 'count': book[1]} for book in books]

# 将结果保存为JSON文件
with open('chart2Data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# 关闭数据库连接
conn.close()


(6, 'Tang')
(15, 'Song')
(16, 'Liao')
(17, 'Jin')
(18, 'Yuan')
(19, 'Ming')
(20, 'Qing')
